In [ ]:
import numpy as np
import pandas as  pd
import matplotlib.pyplot as plt
from warnings import filterwarnings
from mpl_toolkits.mplot3d import Axes3D
import statsmodels.api as sm
import missingno as msno
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.neighbors import LocalOutlierFactor
from scipy.stats import levene
from scipy.stats import shapiro
from scipy.stats.stats import pearsonr
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, ShuffleSplit, GridSearchCV 
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier
from lightgbm import LGBMRegressor, LGBMClassifier
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import tree
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve
import seaborn as sns 
import math

In [ ]:
State_data = pd.read_csv('../input/covid19-in-india/StatewiseTestingDetails.csv')
Cov_data = pd.read_csv('../input/covid19-in-india/covid_19_india.csv')
Vac_data = pd.read_csv('../input/covid19-in-india/covid_vaccine_statewise.csv')

In [ ]:
State_data

In [ ]:
sum(State_data['Negative'].isnull())

In [ ]:
State_data.info()

In [ ]:
State_data['Positive'].max()

In [ ]:
State_data['Positive'].dtype

In [ ]:
main_columns = ['TotalSamples', 'Negative', 'Positive']

In [ ]:
def change_toint(self, column):
    for i, row in enumerate(self[column]):
        try:
            self[column][i] = int(row)
        except:
            continue

In [ ]:
change_toint(State_data, 'Negative')

In [ ]:
def express_positive(self, columns):
    for i, total in enumerate(self[columns[0]]):
        if pd.isna(self[columns[2]][i]) and not pd.isna(self[columns[1]][i]):
            self[columns[2]][i] = total - self[columns[1]][i]
        else:
            continue
    return self

In [ ]:
express_positive(State_data, main_columns)

In [ ]:
State_data.info()

In [ ]:
State_data.drop(['Negative'], axis=1, inplace=True)  #多重共線性の観点より排除(また、Positiveで十分なため)
State_data.dropna(subset=['Positive'], axis=0, inplace=True)

In [ ]:
State_data

In [ ]:
State_data.reset_index(inplace=True)

In [ ]:
def data_separate(self, column):
    self['Year'] = None
    self['Month'] = None
    self['Day'] = None
    for i, time in enumerate(self[column]):
        self['Year'][i] = time.split('-')[0]
        self['Month'][i] = time.split('-')[1]
        self['Day'][i] = time.split('-')[2]
    self[column] = pd.to_datetime(self[column])
    return self

In [ ]:
data_separate(State_data, 'Date')

In [ ]:
State_data.drop(['index'],1,inplace=True)

In [ ]:
for column in ['Year', 'Month', 'Day']:
    State_data[column] = State_data[column].astype('int64')

In [ ]:
State_data.info()

In [ ]:
group_state = State_data.pivot_table(index=['State', 'Year', 'Month', 'Day'], values=['Positive'])

In [ ]:
group_state

In [ ]:
plt.figure(figsize=(15, 8))
plt.plot(State_data['Date'], State_data['TotalSamples'])
plt.xlabel('Time')
plt.ylabel('Num')
plt.title('India Covid')

In [ ]:
State_data['State'].unique()

In [ ]:
"""
def data_visualize(self, column):
    states = self[column].unique()
    plt.subplots_adjust(wspace=0, hspace=0)
    for i, state in enumerate(states):
        df_state = self[self[column] == state]
        x = df_state['Date']
        y1 = df_state['TotalSamples']
        y2 = df_state['Positive']
        #cols = int(math.sqrt(len(states)))
        #rows = len(states)//cols + 1
        cols = 1
        rows = len(states)
        plt.figure(figsize=(10, 10)) # plt.figureは全体のグラフの大きさを定義しているので、forの外に定義すると、subplots全体の大きさになってしまう
        plt.subplot(rows, cols, i+1)
        plt.plot(x,y1, color='blue', marker='v', label='TotalSamples')
        plt.plot(x,y2, color='red', marker='o', label='Positive')
        plt.xlabel('Date', fontsize=18)
        plt.ylabel('Number', fontsize=18)
        plt.title(state, fontsize=18)
        plt.legend(fontsize=18)    # 凡例
        #plt.tick_params(labelsize=18)   # 目盛りの設定
        plt.tight_layout()    # subplotsの間隔を自動調整
    return plt.show()
"""

In [ ]:
def data_visualize(self, column):
    states = self[column].unique()
    #plt.subplots_adjust(wspace=0, hspace=0)
    for i, state in enumerate(states):
        df_state = self[self[column] == state]
        x = df_state['Date']
        y1 = df_state['TotalSamples']
        y2 = df_state['Positive']
        #cols = int(math.sqrt(len(states)))
        #rows = len(states)//cols + 1
        cols = 1
        rows = len(states)
        plt.figure(figsize=(15, 10)) # plt.figureは全体のグラフの大きさを定義しているので、forの外に定義すると、subplots全体の大きさになってしまう
        #plt.subplot(rows, cols, i+1)
        plt.plot(x,y1, color='blue', marker='v', label='TotalSamples')
        plt.plot(x,y2, color='red', marker='o', label='Positive')
        plt.xlabel('Date', fontsize=18)
        plt.ylabel('Number', fontsize=18)
        plt.title(state, fontsize=18)
        plt.legend(fontsize=18)    # 凡例
        #plt.tick_params(labelsize=18)   # 目盛りの設定
        #plt.tight_layout()    # subplotsの間隔を自動調整
        plt.show()

In [ ]:
data_visualize(State_data, 'State')

In [ ]:
def percentage_visualize(self, column):
    states = self[column].unique()
    for state in states:
        df_data = self[self[column] == state]
        x = df_data['Date']
        y = (df_data['Positive'] / df_data['TotalSamples'])*100
        plt.figure(figsize=(10,8))
        plt.plot(x, y, color='black')
        plt.xlabel('Date', fontsize=18)
        plt.ylabel('Percentage  %', fontsize=18)
        plt.title(state, fontsize=18, color='red')
        plt.ylim(0,20)
        plt.show()

In [ ]:
percentage_visualize(State_data, 'State')

In [ ]:
Cov_data

In [ ]:
C_data = Cov_data.copy()

In [ ]:
C_data.drop(['Time','Sno','ConfirmedIndianNational', 'ConfirmedForeignNational'], axis=1, inplace=True)

In [ ]:
C_data

In [ ]:
S_data = State_data.copy()

In [ ]:
C_data.info()

In [ ]:
C_data['Date'] = pd.to_datetime(C_data['Date'])

In [ ]:
def total_num_at_point(self, column):
    self['TotalCured'] = None   #コラムを作っておく
    self['TotalDeaths'] = None
    self['TotalConfirmed'] = None
    states = self[column].unique()
    df = pd.DataFrame()
    for state in states:
        df_data = self[self[column] == state].reset_index(drop=True)
        Total_cured = 0
        Total_deaths = 0
        Total_Confirmed = 0
        for i, row in enumerate(df_data.iterrows()):      # iterrows: 一行ごとに取り出す
            Total_cured += row[1][2]          #Series化するといろいろとめんどくさい(columnで指定できなくなるため,row[1][columnの順番]で指定)(row[0]だとindexが出てくる)
            Total_deaths += row[1][3]    #3でcolumnのcuredを指名
            Total_Confirmed += row[1][4]      #4でcolumnのDeathsを指定
            df_data['TotalCured'][i] = Total_cured
            df_data['TotalDeaths'][i] = Total_deaths
            df_data['TotalConfirmed'][i] = Total_Confirmed
        df = df.append(df_data)
    ordered_data = df.reset_index(drop=True)
    self = pd.merge(self, ordered_data, left_on=['Date', 'State/UnionTerritory', 'Cured', 'Deaths', 'Confirmed'],
                              right_on=['Date', 'State/UnionTerritory', 'Cured', 'Deaths', 'Confirmed'], how='inner')
    self.drop(['TotalCured_x','TotalDeaths_x','TotalConfirmed_x'], axis=1, inplace=True)
    self.rename(columns={'TotalCured_y':'TotalCured', 'TotalDeaths_y': 'TotalDeaths', 'TotalConfirmed_y':'TotalConfirmed'}, inplace=True)
    return self

In [ ]:
C_data_a = total_num_at_point(C_data, 'State/UnionTerritory')

In [ ]:
def total_num_at_point_df(self, column):
    self['TotalCured'] = None   #コラムを作っておく
    self['TotalDeaths'] = None
    self['TotalConfirmed'] = None
    states = self[column].unique()
    df = pd.DataFrame()
    for state in states:
        df_data = self[self[column] == state].reset_index(drop=True)  #drop=True,で元のインデックスをコラムに挿入しないようにする
        Total_cured = 0
        Total_deaths = 0
        Total_Confirmed = 0
        for i, row in enumerate(df_data.iterrows()):      # iterrows: 一行ごとに取り出す
            Total_cured += row[1][2]          #Series化するといろいろとめんどくさい(columnで指定できなくなるため,row[1][columnの順番]で指定)(row[0]だとindexが出てくる)
            Total_deaths += row[1][3]    #3でcolumnのcuredを指名
            Total_Confirmed += row[1][4]      #4でcolumnのDeathsを指定
            df_data['TotalCured'][i] = Total_cured   #[x:y]はインデックスに関係ないスライスで取り出していて、[z]はindexを指定して取り出していることに注意
            #print('{}/{}'.format(i, df_data['TotalCured'][i]))
            df_data['TotalDeaths'][i] = Total_deaths
            df_data['TotalConfirmed'][i] = Total_Confirmed
        df = df.append(df_data)
    ordered_data = df.reset_index(drop=True)
    #self = pd.merge(self, ordered_data, left_on=['Date', 'State/UnionTerritory', 'Cured', 'Deaths', 'Confirmed'],
                              #right_on=['Date', 'State/UnionTerritory', 'Cured', 'Deaths', 'Confirmed'], how='inner')
    return ordered_data

In [ ]:
ordered_C_data = total_num_at_point_df(C_data, 'State/UnionTerritory')

In [ ]:
C_data_a

In [ ]:
ordered_C_data.info()

In [ ]:
ordered_C_data

In [ ]:
"""
def visualize_covid(self, column):
    states = self[column].unique()
    for state in states:
        s_data = self[self[column] == state]
        x = s_data['Date']
        y1 = s_data['TotalCured']
        y2 = s_data['TotalDeaths']
        y3 = s_data['TotalConfirmed']
        plt.figure(figsize=(10,8))
        plt.plot(x,y1,color='blue',label='TotalCured')
        plt.plot(x,y2,color='black',label='TotalDeaths')
        plt.plot(x,y3,color='red',label='TotalConfirmed')
        plt.xlabel('Date',fontsize=18)
        plt.ylabel('Num',fontsize=18)
        plt.legend(fontsize=18)
        plt.title(state, fontsize=18, color='red')
        plt.ylim(0,self['TotalConfirmed'].max())
        plt.show()
"""

In [ ]:
visualize_covid(ordered_C_data, 'State/UnionTerritory')

In [ ]:
ordered_C_data = ordered_C_data[ordered_C_data['TotalConfirmed'] != 0]  # zerodivisionerrorを起こさないため

In [ ]:
"""
def visualize_deaths_percentage(self,column):
    states = self[column].unique()
    for state in states:
        df_data = self[self[column] == state]
        x = df_data['Date']
        y = df_data['TotalDeaths']/df_data['TotalConfirmed']*100
        plt.figure(figsize=(10,8))
        plt.bar(x,y,color='lightblue', label='TotalConfirmed  {}'.format(df_data['TotalConfirmed'].max()))
        plt.ylabel('Percentage of Deaths  %', fontsize=18)
        plt.xlabel('Date', fontsize=18)
        plt.ylim(0,10)
        plt.title(state, fontsize=18, color='red')
        plt.legend(fontsize=18)
        plt.show()
"""

In [ ]:
ordered_C_data['TotalConfirmed'].sum()

In [ ]:
#visualize_deaths_percentage(ordered_C_data, 'State/UnionTerritory')

In [ ]:
# totalを間違って作ってしまったので、この前は無視

In [ ]:
def visualize_covid2(self, column):
    states = self[column].unique()
    for state in states:
        s_data = self[self[column] == state]
        x = s_data['Date']
        y1 = s_data['Cured']
        y2 = s_data['Deaths']
        y3 = s_data['Confirmed']
        plt.figure(figsize=(10,8))
        plt.plot(x,y1,color='blue',label='TotalCured')
        plt.plot(x,y2,color='black',label='TotalDeaths')
        plt.plot(x,y3,color='red',label='TotalConfirmed')
        plt.xlabel('Date',fontsize=18)
        plt.ylabel('Num',fontsize=18)
        plt.legend(fontsize=18)
        plt.title(state, fontsize=18, color='red')
        plt.ylim(0,self['Confirmed'].max())
        plt.show()
        
        plt.figure(figsize=(10,8))
        dx = np.gradient(x.values.astype('datetime64[D]').astype('int64'))   # 日ごとのdatetimeに変更して微分する(dataframeではDで保存できないため、valuesで変換)、その後int型に変換
        dy = np.gradient(y3)
        slope = dy/dx
        #or slope = np.gradient(y3, x.values.astype('datetime64[D]'))
        plt.plot(x, slope, color='green', marker='v')
        plt.xlabel('Date', fontsize=18)
        plt.ylabel('Gradient of Configured', fontsize=18)
        plt.title(state, fontsize=18, color='green')
        plt.show()

In [ ]:
def visualize_deaths_percentage(self,column):
    states = self[column].unique()
    for state in states:
        df_data = self[self[column] == state]
        x = df_data['Date']
        y = df_data['Deaths']/df_data['Confirmed']*100
        plt.figure(figsize=(10,8))
        plt.bar(x,y,color='lightblue', label='TotalConfirmed  {}'.format(df_data['TotalConfirmed'].max()))
        plt.ylabel('Percentage of Deaths  %', fontsize=18)
        plt.xlabel('Date', fontsize=18)
        plt.ylim(0,10)
        plt.title(state, fontsize=18, color='red')
        plt.legend(fontsize=18)
        plt.show()

In [ ]:
visualize_covid2(ordered_C_data, 'State/UnionTerritory')

In [ ]:
visualize_deaths_percentage2(ordered_C_data, 'State/UnionTerritory')

In [ ]:
y = [100000, 102000, 130000]
np.gradient(y)

In [ ]:
np.gradient(ordered_C_data['Date'])

In [ ]:
ordered_C_data['Date'].astype('datetime64[D]')

In [ ]:
ordered_C_data['Date'].values.astype('datetime64[D]')   # values でdataframeやseriesからnp.array化して初めてnsからDにすることができる(dataframe,seriesではnsのみ保存できる)

In [ ]:
ordered_C_data['Date'].values.astype('datetime64[D]').astype('int64')